# Imports

In [2]:
import os
import cv2
import numpy as np
import pandas as pd

from insightface.app import FaceAnalysis

# Configure Model

In [3]:
# buffalo_l model

model_l = FaceAnalysis(name='buffalo_l',
                        root='/Users/boss/Desktop/Notes/4_attendance_app/insightface_model/models/buffalo_l', 
                        providers=['CUDAExecutionProvider'])

model_l.prepare(ctx_id=0, det_size=(640, 640), det_thresh=0.5) # input size

/Users/boss/miniconda3/envs/tensorflow/lib/python3.10/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:69: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/boss/Desktop/Notes/4_attendance_app/insightface_model/models/buffalo_l/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/boss/Desktop/Notes/4_attendance_app/insightface_model/models/buffalo_l/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/boss/Desktop/Notes/4_attendance_app/insightface_model/models/buffalo_l/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/boss/Desktop/Notes/4_attendance_app/insightface_model/models/buffalo_l/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.

# Data Preparation

## Extract Facial Fetaures and Labels

In [33]:
images_dir = '/Users/boss/Desktop/Notes/2_Fast_Face_Recognition_System/images'
list_dir = os.listdir(path=images_dir)
print(list_dir)

['teacher-Barack_Obama', 'student-Angelina_Jolie', 'student-Chris_Evans', 'teacher-Morgan_Freeman', 'student-Scarlett_Johansson']


In [25]:
import re

def remove_special_characters(input_string):
    # Define a regular expression pattern to match special characters
    pattern = r'[^a-zA-Z0-9\s]'  # This pattern matches anything that is not a letter, digit, or whitespace

    # Use re.sub() to replace matched special characters with an empty string
    result_string = re.sub(pattern, ' ', input_string)

    return result_string.title()

In [41]:
# # extracting names from listed directories
# role_and_name = {remove_special_characters(name.split('-')[1]): remove_special_characters(name.split('-')[0]) for name in list_dir if '-' in name}
# # print(role_and_name)

person_info = []
for folder in list_dir:
    role, name = folder.split('-')
    name = remove_special_characters(name)
    role = remove_special_characters(role)
    img_files = os.listdir(path=f'{images_dir}/{folder}')
    # print(img_files)
    for file in img_files:
        path = f'{images_dir}/{folder}/{file}'
        # print(path)

        # read image
        img = cv2.imread(path)

        # get info
        result = model_l.get(img, max_num=1)
        
        if len(result) > 0:
            # get embeddings
            res = result[0]
            embeddings = res['embedding']

            # appending to list
            person_info.append([name, role, embeddings])
            
            
print(person_info)

        

/Users/boss/miniconda3/envs/tensorflow/lib/python3.10/site-packages/insightface/utils/transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4


[['Barack Obama', 'Teacher', array([ 1.01677813e-01,  8.72059345e-01,  8.20790976e-02,  8.66239786e-01,
       -6.83377326e-01, -5.86362481e-01, -5.17531157e-01,  3.50272149e-01,
       -8.00685823e-01,  1.59926295e+00,  3.13717335e-01, -1.24295270e+00,
       -3.54073972e-01, -9.94484484e-01, -1.40921688e+00,  5.42272806e-01,
       -2.14447320e-01, -5.45962214e-01, -3.98445219e-01, -1.88824296e-01,
        8.36232722e-01, -3.89840752e-02,  6.32570684e-02,  1.92688197e-01,
        9.23383653e-01, -3.12970102e-01,  4.67518687e-01,  5.51410079e-01,
       -6.14781797e-01,  4.30935442e-01, -9.02108192e-01,  2.16013223e-01,
       -1.18549049e+00, -2.14485690e-01,  1.35073602e-01, -8.47282946e-01,
       -9.25076753e-02,  6.97160959e-02, -1.48303461e+00,  7.33820200e-01,
       -2.19972587e+00,  1.17768252e+00,  3.40460092e-02,  3.37222308e-01,
        1.71789765e+00,  4.29728687e-01,  1.33849812e+00, -1.78957254e-01,
       -5.22468448e-01,  1.13050306e+00,  4.80523519e-02, -3.94423544e-

In [46]:
# conversion to dataframe
df = pd.DataFrame(person_info, columns=['Name', 'Role', 'Facial_Features'])
df.sample(5) # store in Redis

,Name,Role,Facial_Features
43,Scarlett Johansson,Student,"[-0.06889759, -0.67759174, 0.5657509, 0.206604..."
48,Scarlett Johansson,Student,"[-0.9165662, -1.0052961, -0.8792947, -0.008987..."
20,Chris Evans,Student,"[0.54855955, -0.48537457, -0.03267619, -0.7943..."
30,Morgan Freeman,Teacher,"[0.75762725, 0.5655479, 0.29826075, -0.7784413..."
33,Morgan Freeman,Teacher,"[0.953475, 1.0518413, 1.0741951, 0.021500979, ..."


## Inference Testing

In [50]:
img_test = cv2.imread('/Users/boss/Desktop/Notes/2_Fast_Face_Recognition_System/test_images/test_2.jpeg')

# feature extraction 

test_results = model_l.get(img_test, max_num=0)

for res in test_results:
    bbox = res['bbox']
    score = res['det_score']
    test_embedding = res['embedding']

In [51]:
len(test_embedding)

512